# TODO
- vP is not allow to be 0 or 1
- DONE add everything to dictionaries
- DONE attacker only works with dict
- optimise
- Max it is dependent on deltaB
- DONE t must not be prime --> optimise
- DONE try catch when non invertible prime
- DONE when there is no R large/ small en
- DONE when delta of bounds is small then the possibility of a new generation of p is high 
- DONE when p is smaller than there is is higher chance of success
- Problem when m>=n 
- Use secret module instead of the random module for cryptographic purposes
- DONE camelCase to snake_case if required
- gen_prime() is not used
- Check if N/P is not a hole number

### Import and Configuration

In [56]:
import sympy
import random
import coloredlogs, logging
import math

In [57]:
coloredlogs.install(fmt='%(levelname)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [58]:
MIN_PRIME = 0
MAX_PRIME = 10
OPT_RANGE_REDUCTION = 0

MAX_ITERS = 10

msg = 12345678900987654321

In [59]:
def gen_prime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def gen_prime_with_optimized_bit_limit():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def gen_prime_with_boundaries(lb, ub):
    return sympy.randprime(lb, ub)

def gen_prime_half_size():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def gen_random_number_with_bit_limit():
    return random.randint(0, pow(2, MAX_PRIME))

def fermat_prime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def modular_multiplicative_inverse(E, phiN):
    return pow(E, -1, phiN)

def calc_n(P, Q):
    return P*Q

def calc_phi_n(P, Q):
    return (P-1) * (Q-1)

def to_binary(dec):
    return "{0:b}".format(dec)

def to_decimal(bin):
    return int(bin, 2)

def concatenate_in_binary(prev, tail):
    prev = to_binary(prev)
    tail = to_binary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return to_decimal(prev + tail)

def split_in_binary(bin):
    bin = to_binary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(to_decimal(prev), to_decimal(tail))


# Voraussetzungen

In [60]:
def create_attacker_key():
    a_P = gen_prime_half_size()
    a_Q= gen_prime_half_size()


    a_PhiN = calc_phi_n(a_P, a_Q)
    a_N = calc_n(a_P, a_Q)

    a_E = fermat_prime()
    a_D = modular_multiplicative_inverse(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

attacker = {}

created = False
while not created:
    try:
        attacker = create_attacker_key()
        created = True
    except ValueError:
            logger.error("Trying to generate Key failed. Trying again ...")
logger.info("Attacker Key successfully generated!")

INFO Attacker Key successfully generated!


# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [61]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
user = {}

valid = False
while not valid:
    # Schritt 1
    P = gen_prime_with_optimized_bit_limit()
    vP = pow(P, attacker["E"] ,attacker["N"])

    lower_bound_Q = math.ceil(concatenate_in_binary((vP-1), 0)/P)
    upper_bound_Q = math.floor(concatenate_in_binary(vP, (pow(2, MAX_PRIME)-2))/P)

    

    try:
        Q = int(gen_prime_with_boundaries(lower_bound_Q, upper_bound_Q))
        valid = True

        print("P: ",P)
    
        print("Lower: ",lower_bound_Q)
        print("Upper: ",upper_bound_Q)
    except ValueError:
        logger.error("Margain to small")
        pass


    # # Schritt 2
    # # t = gen_random_number_with_bit_limit() 
    # t = pow(2, MAX_PRIME+1) -1
    # tempN = concatenate_in_binary(vP, t)
    
    # # Schritt 3
    # # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    # ub = int(int(tempN)//int(P)) 
    # lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    # # logger.info("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    # R = 0
    # Q = 0
    # try:
    #     for i in range(MAX_ITERS):
    #         Q = int(gen_prime_with_boundaries(lb, ub))
    #         R = tempN - P*Q

    #         if (R < pow(2,MAX_PRIME+1) and R >= 0):
    #             valid = True
    #             logger.info("Iterations: {0}".format(i))
    #             break
    #         r_too_large += 1
    # except ValueError:
    #     pass

    
    user["P"] = P
    user["Q"] = Q

attacker["vP"] = vP
# attacker["R"] = R

logger.info("Amount of failed attempts: {0}".format(r_too_large))

logger.info("tempN = P * Q + R")
# logger.info("{0} = {1} * {2} + {3}".format(tempN, P, Q, R))

INFO Amount of failed attempts: 0
INFO tempN = P * Q + R


P:  853
Lower:  466
Upper:  468


## Schritt 4

In [62]:
user["N"] = calc_n(user["P"], user["Q"])

## Schritt 5

In [63]:
def create_user_key():
    PhiN = calc_phi_n(P, Q)
    E = fermat_prime()
    D = modular_multiplicative_inverse(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        user.update(create_user_key())
        created = True
    except ValueError:
            logger.info("Trying to generate Key failed. Trying again ...")
logger.info("User Key successfully generated!")

INFO User Key successfully generated!


## Schritt 6

In [64]:
cipher = pow(msg, user["E"], user["N"])
signature = pow(msg, user["D"], user["N"])

# Angriff

In [65]:
class Calc_Secret_Result():
    was_successfull = True
    key_parameter = {}

In [66]:
def calc_secrets(vP):
    calc_secret_result = Calc_Secret_Result()

    P = pow(vP, attacker["D"], attacker["N"])
    # Potential to optimize when N/P not hole Number
    Q =  int(pub_user["N"]//P)
    PhiN =  int(calc_phi_n(P, Q))
    try:
        D = modular_multiplicative_inverse(pub_user["E"], PhiN)
        calc_secret_result.key_parameter = {
        "P" : P,
        "Q" : Q,
        "PhiN" : PhiN,
        "D" : D, 
    }
    except ValueError:
        calc_secret_result.was_successfull = False
        logger.info("Trying to generate Key failed. Trying again ...")
    
    return calc_secret_result

In [67]:
def validate_through_signature(key_parameter):
    key_parameter["Sig"] = pow(msg, key_parameter["D"], collection_user["N"])
    if (key_parameter["Sig"] == signature):
        collection_user.update(key_parameter)
        return True
    return False
    

## Schritt 1

In [68]:
pub_user = {
    "N" : user["N"],
    "E" : user["E"]
}

collection_user = {
    "N" : pub_user["N"],
    "E" : pub_user["E"]
}

## Schritt 2

In [69]:
collection_user["vP"] = split_in_binary(user["N"])[0]
collection_user["vP'"] = collection_user["vP"]+1

In [70]:
secrets_for_vP = calc_secrets(collection_user["vP"])
secrets_for_vP_plus_one = calc_secrets(collection_user["vP'"])

if(secrets_for_vP.was_successfull and secrets_for_vP_plus_one.was_successfull):
    logger.info("Both key parameters of D were successfully calculated.")
    if(validate_through_signature(secrets_for_vP.key_parameter)):
        logger.info("D is the right privat Key: {0}".format(collection_user["D"]))
    elif(validate_through_signature(secrets_for_vP_plus_one.key_parameter)):
        logger.info("D' is the right privat Key: {0}".format(collection_user["D"]))
elif(not secrets_for_vP.was_successfull and not secrets_for_vP_plus_one.was_successfull):
    logger.error("Both key parameters calculated with errors!")
elif(not secrets_for_vP_plus_one.was_successfull):
    logger.info("Second key parameters calculated with error! First key parameters must be correct!")
    if(validate_through_signature(secrets_for_vP.key_parameter)):
        logger.info("D is the right privat Key: {0}".format(collection_user["D"]))
elif(not secrets_for_vP.was_successfull):
    logger.info("First key parameters calculated with error! Second key parameters must be correct!")
    if(validate_through_signature(secrets_for_vP_plus_one.key_parameter)):
        logger.info("D' is the right privat Key: {0}".format(collection_user["D"]))
else:
    logger.error("Logical programming error!")


INFO Trying to generate Key failed. Trying again ...
INFO Second key parameters calculated with error! First key parameters must be correct!


# Result

In [71]:
def sort_dic(dict):
    return {key: dict[key] for key in sorted(dict)}

In [72]:
# logger.info(sort_dic(attacker))
# logger.info(sort_dic(user))
# logger.info(sort_dic(collection_user))

# Validation Tests

### Validation test for vP

In [73]:
testVar = "vP"
altTestVar = "vP'"

if(collection_user[testVar] == attacker[testVar] or collection_user[altTestVar] == attacker[testVar]):
    logger.info("Success!!!")
    logger.info("The variable {0} or {1} is in the expected state: {2}".format(testVar, altTestVar, attacker[testVar]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(attacker[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variable vP or vP' is in the expected state: 389
INFO Current vP: 389 	 Current vP': 390


### Validation test for P and P'

In [74]:
testVar = "P"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state:".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], collection_user[testVar]))

KeyError: 'P'

### Validation test for Q and Q'

In [ ]:
testVar = "Q"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state!".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], attacker[testVar]))

INFO Success!!!
INFO The variables Q are in the expected state!
INFO Current Q: 532000395873127143997729716885179821652078618902636638732404060344144589237346651801417587359542834416047911099194755802907278745709944499503518333307358346344132084833681887116211102872670630552811149282224070801062838605508432627045389750341602303691886298851773738623270555639748182972262549646832578540177


### Validation test for PhiN and PhiN'

In [ ]:
testVar = "PhiN"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state:".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variables PhiN are in the expected state:
INFO Current PhiN: 69680211169244889098966824008678441984600435738992785020107746848202918275383162564586507675041251494518942607793632771724909614545290479092864343551697789302448544931566618731436913547880166659262504263927823399113215587986814429019300708430875920571403462925795613442010427027459056907093940845516390348481408836757528681225114384339601448212957972192858891961389619077407139351135978249826438161667653745363238032485526116289501665857913671854535484573968047572183247613668080854613611956873553127147490659408238421243308625625399096056565354182178439320655433321347904176269964104997799405783536859654231240608


### Validation test for D and D'

In [ ]:
testVar = "D"
altTestVar = "D'"

if(collection_user[testVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} are in the expected state:".format(testVar))
    logger.info("Current {0}: {1}".format(testVar, collection_user[testVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(user[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variables D are in the expected state:
INFO Current D: 41808126701546933459380094405207065190760261443395671012064648108921750965229897538751904605024750896711365564676179663034945768727174287455718606131018673581469126958939971238862148128728099995557502558356694039467929352792088657411580425058525552342842077755477368065206256216475434144256364507309834209088845302054517208735068630603760868927774783315715335176833771446444283610681586949895862897000592247217942819491315669773700999514748203112721290744380828543309948568200848512768167174124131876288494395644943052745985175375239457633939212509307063592393259992808742505761978462998679643470122115792538744365


### Validation test for signature and signature'

In [ ]:
testVar = "Sig"

if(collection_user[testVar] == signature):
    collection_user["correct_key"] = "D"
    logger.info("D is the right privat Key: {0}".format(collection_user[testVar]))
else:
    logger.error("Message was incorrectly signed!")

INFO D is the right privat Key: 57237726590858504813250198664048193274159749055985008043621584876466360217570112171939107963168467914223148559434413924458084876519949177276881934173090727446247923846521126313296148125540962890445706295040989488650041925218241421542960643124756004161344512948707495317471341466810432273018976027311830163518697229949626875216502310950266817216906596873660372715940285868204965502453904142803781803614296794806332879873227196524759493234560939236755023591729071376982335033879292629478352530642007765706720326153195104516685392108765050675285799459873243066639856300219325330696766961641428857578937479392872861634
